In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
pd.options.plotting.backend = "plotly"

import sys
sys.path.insert(0, 'driftpy/src/')
import driftpy
# print(driftpy.__file__)

import numpy as np 
from sim.sim import DriftSim, load_hist_oracle

In [ ]:
# sqrt_k = 100000
# d = -28.789
# Q = 50
# x = sqrt_k - d
# y = (sqrt_k**2)/x
# print(d/x)

# def cost_of_pk(p):
#     cost = -((1 / (x + d) - p / (x * p + d)) * y * d * Q)
#     return cost

# def cost_of_pk2(p):
#     cost = (d**2 * (p-1)*Q*y)/(p*x**2)
#     return cost

# scale = 1000

# df = pd.DataFrame([cost_of_pk(float(x)/scale) for x in range(scale - int(scale/20), scale + int(scale/20), 1)])
# df.index /= scale
# df.index += .95
# df[1] = [cost_of_pk2(p) for p in df.index]
# df.plot()

In [ ]:

# def pk_of_cost(C):
#     numer = y * d * d * Q - C * d * (x + d)
#     denom = C * x * (x + d) + y * d * d * Q
#     # print(C, x, y, d, Q)
#     # print(numer, denom)
#     p = numer / denom
#     return p

# def pk_of_cost2(C):
#     if C!=0:
#         return (-d/x + (d*d*y*Q)/(C*x*x))
#     else:
#         return 1
    
# def pk_for_cost3(C):
#     pk = 1 + C*x/(Q*d*d)
#     return pk

# scale = int(1e3)
# df = pd.DataFrame([pk_of_cost(-float(x)/1e6) for x in range(int(-scale*100), 
#                                                              int(scale*100), 
#                                                              scale)])
# df.index -= 100
# df.index *= scale/1e6

# df[1] = [pk_for_cost3(p) for p in df.index]
# (df).plot()

In [ ]:
def prepeg_budget_k(deficit):
    assert(deficit<0)
    # thinks a lower k (p) is required to achieve cost saving (thus underestimates pk)
    pk_delta = C*x/(Q*d*d) 
    if pk_delta < .025:
        pk = .975
    elif pk_delta < 0:
        pk = 1 + pk_delta
    else:
        assert(False)
    return pk

In [ ]:
# ((x+d)**2)/(d*d*Q*y)
# p = .98
# (d**2 * (p-1)*Q*y)/(p*x**2)

In [ ]:
import os
import datetime
from sim.agents import * 
from programs.clearing_house.state import * 
from sim.events import OpenPositionEvent
from sim.helpers import random_walk_oracle, rand_heterosk_oracle, class_to_json
import pickle as cPickle

SIM_NAME = 'sim-results/sim-solhist'
# load_hist_oracle('LUNA-PERP', 'sim-solhist/oracle_prices.csv')

def make_ch(base_spread, strategies=''):
    oracle = Oracle(SIM_NAME+'/oracle_prices.csv')
    # oracle.to_csv(SIM_NAME+'/oracle_prices.csv')
    amm = AMM(
        oracle=oracle, 
        base_asset_reserve=int(367621.62052551797 * 1e13), 
        quote_asset_reserve=int(367621.62052551797 * 1e13),
        funding_period=60*60,
        peg_multiplier=int(oracle.prices[0]*1e3),
        base_spread = base_spread,
        strategies = strategies
    )
    market = Market(amm)
    
    if strategies=='':
        strategies = 'v1'
    ch_name = strategies+'_'+str(base_spread)
    
    fee_structure = FeeStructure(numerator=1, denominator=1000)
    clearing_house = ClearingHouse([market], fee_structure)
    clearing_house.name = ch_name

    return clearing_house


arb_agent1 = Noise(1, 0, 0, lookahead=0, size=10)
arb_funding1 = ArbFunding(1, 0, 1,)
agents = [arb_agent1, arb_funding1]

sim1 = DriftSim(SIM_NAME, clearing_house=make_ch(0), agents=agents)
sim2 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, ''), agents=agents)
sim2 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, 'PrePeg'), agents=agents)
sim3 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, 'InventorySkew'), agents=agents)
sim4 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, 'VolatilityScale'), agents=agents)
sim5 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, 'OracleRetreat'), agents=agents)
sim6 = DriftSim(SIM_NAME, clearing_house=make_ch(1e3, 'PreFreePeg_InventorySkew_OracleRetreat'),
                agents=agents)


sims = [sim1, sim2, sim3, sim4, sim5, sim6]
sims = [sim6]

for sim_i in sims:
    res = sim_i.run()
    print('finish sim')
    res_df = sim_i.to_df(save=True)
    print('finish res')
    print(res_df.columns)
    toplt = res_df[[
        'm0_mark_price', 'm0_oracle_price', 'm0_bid_price', 'm0_ask_price',
        # 'bid_price_before', 'ask_price_before',
        # 'user0.quote_asset_amount',
        # 'cumulative_funding_rate_long',
        'm0_peg_multiplier', 'm0_wouldbe_peg', 'm0_wouldbe_peg_cost',
        'm0_mark_std',
        # 'm0_total_exchange_fees', 'm0_total_mm_fees',
        # 'm0_total_fee', 'm0_total_fee_minus_distributions',


        # 'base_asset_amount',
        # 'user0.base_asset_amount', 'user0.total_collateral', 'mark_std', 'last_spread',
        # 'total_fees', 
        # 'predicted_long_funding',
        # 'total_exchange_fees', 'total_mm_fees'
    ]].replace(0, np.nan)
    if 'm0_net_base_asset_amount' in res_df.columns:

        toplt = pd.concat([toplt, res_df[['m0_net_base_asset_amount']]],axis=1)
        toplt['m0_net_base_asset_amount']/=1e13
        # toplt['user0.quote_asset_amount']/=1e6
    toplt['m0_peg_multiplier']/=1e3

    # toplt['base_asset_amount']/=1e13
    fig = toplt.plot(title=sim_i.clearing_house.name)
    fig.show()

examine market information

In [ ]:
(res_df['m0_peg_multiplier']/1e3).rolling(60*60).std().plot()

In [ ]:
(res_df[['m0_total_fee', 'm0_total_fee_minus_distributions', 'm0_total_exchange_fees']]).plot()

In [ ]:
(res_df['m0_quote_asset_reserve'].astype(int)/res_df['m0_base_asset_reserve'].astype(int)).plot()

In [ ]:
res_df[['m0_base_asset_reserve', 'm0_quote_asset_reserve', 'm0_sqrt_k',]].astype(int).plot()

run rust impl simulation using events.csv from python sim

In [ ]:
# ! cd protocol-v1/ && bash stress/run_local_sim.sh ../sim-results/sim-solhist/chPrepeg_1000.0

examine user information

In [ ]:
user = 'u0'
(res_df[[user+'_'+x for x in ['collateral',
       # 'u0_m0_upnl', 
         'total_collateral',
       'm0_ufunding',
       'm0_base_asset_amount',
         # 'user0.free_collateral',
       # 'user0.margin_ratio', 'user0.total_position_value'
       ]]]/1e6 - 10e6).plot()

In [ ]:
entry_price =((res_df['u0_m0_quote_asset_amount']*1e7)/res_df['u0_m0_base_asset_amount'])
pd.concat([entry_price, res_df['m0_mark_price'],
           res_df['m0_ask_price'],
           res_df['m0_bid_price']
          ],axis=1).plot()

examine funding rate information

In [ ]:
print(res_df.columns)
r = res_df[['m0_last_oracle_price_twap',
       'm0_last_mark_price_twap',
        'm0_last_bid_price_twap',
            'm0_last_ask_price_twap',
       'm0_last_mid_price_twap',
       ]]
# r = res_df[['oracle_price',
#        'mark_price',
#         'bid_price',
#             'ask_price',
#        ]]
# r['mid_price'] = (r['bid_price']+r['ask_price'])/2
r.plot()

In [ ]:
(res_df[['m0_cumulative_funding_rate_long']]/1e6).plot()

In [ ]:
res_df[['m0_predicted_long_funding','m0_predicted_short_funding']].plot()